In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [2]:
!pip install -qU rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 48.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors, Crippen, Descriptors3D, MACCSkeys
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem.Scaffolds import MurckoScaffold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.base import clone
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import inspect

train = pd.read_csv('/kaggle/input/melting-point/train.csv')
test = pd.read_csv('/kaggle/input/melting-point/test.csv')
sample_sub = pd.read_csv('/kaggle/input/melting-point/sample_submission.csv')

def extract_molecular_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return {f'feat_{i}': np.nan for i in range(100)}
    
    features = {}
    
    features['MolWt'] = Descriptors.MolWt(mol)
    features['MolLogP'] = Descriptors.MolLogP(mol)
    features['NumHDonors'] = Descriptors.NumHDonors(mol)
    features['NumHAcceptors'] = Descriptors.NumHAcceptors(mol)
    features['NumRotatableBonds'] = Descriptors.NumRotatableBonds(mol)
    features['NumAromaticRings'] = Descriptors.NumAromaticRings(mol)
    features['NumAliphaticRings'] = Descriptors.NumAliphaticRings(mol)
    features['RingCount'] = Descriptors.RingCount(mol)
    features['TPSA'] = Descriptors.TPSA(mol)
    features['NumHeteroatoms'] = Descriptors.NumHeteroatoms(mol)
    features['BertzCT'] = Descriptors.BertzCT(mol)
    features['HallKierAlpha'] = Descriptors.HallKierAlpha(mol)
    features['Kappa1'] = Descriptors.Kappa1(mol)
    features['Kappa2'] = Descriptors.Kappa2(mol)
    features['Kappa3'] = Descriptors.Kappa3(mol)
    features['NumValenceElectrons'] = Descriptors.NumValenceElectrons(mol)
    features['NumRadicalElectrons'] = Descriptors.NumRadicalElectrons(mol)
    features['LabuteASA'] = Descriptors.LabuteASA(mol)
    features['NumAtoms'] = mol.GetNumAtoms()
    features['NumHeavyAtoms'] = mol.GetNumHeavyAtoms()
    features['NumBonds'] = mol.GetNumBonds()
    features['FormalCharge'] = Chem.GetFormalCharge(mol)
    
    try:
        features['FractionCsp3'] = Descriptors.FractionCSP3(mol)
    except:
        features['FractionCsp3'] = 0
    
    features['MolMR'] = Descriptors.MolMR(mol)
    features['Chi0v'] = Descriptors.Chi0v(mol)
    features['Chi1v'] = Descriptors.Chi1v(mol)
    features['Chi2v'] = Descriptors.Chi2v(mol)
    features['Chi3v'] = Descriptors.Chi3v(mol)
    features['Chi4v'] = Descriptors.Chi4v(mol)
    
    try:
        features['BalabanJ'] = Descriptors.BalabanJ(mol)
    except:
        features['BalabanJ'] = 0
    
    features['MaxEStateIndex'] = Descriptors.MaxEStateIndex(mol)
    features['MinEStateIndex'] = Descriptors.MinEStateIndex(mol)
    features['NumAliphaticCarbocycles'] = Descriptors.NumAliphaticCarbocycles(mol)
    features['NumAliphaticHeterocycles'] = Descriptors.NumAliphaticHeterocycles(mol)
    features['NumAromaticCarbocycles'] = Descriptors.NumAromaticCarbocycles(mol)
    features['NumAromaticHeterocycles'] = Descriptors.NumAromaticHeterocycles(mol)
    features['NumSaturatedCarbocycles'] = Descriptors.NumSaturatedCarbocycles(mol)
    features['NumSaturatedHeterocycles'] = Descriptors.NumSaturatedHeterocycles(mol)
    
    try:
        features['Asphericity'] = Descriptors3D.Asphericity(mol)
        features['Eccentricity'] = Descriptors3D.Eccentricity(mol)
        features['InertialShapeFactor'] = Descriptors3D.InertialShapeFactor(mol)
        features['NPR1'] = Descriptors3D.NPR1(mol)
        features['NPR2'] = Descriptors3D.NPR2(mol)
        features['RadiusOfGyration'] = Descriptors3D.RadiusOfGyration(mol)
        features['SpherocityIndex'] = Descriptors3D.SpherocityIndex(mol)
    except:
        features['Asphericity'] = 0
        features['Eccentricity'] = 0
        features['InertialShapeFactor'] = 0
        features['NPR1'] = 0
        features['NPR2'] = 0
        features['RadiusOfGyration'] = 0
        features['SpherocityIndex'] = 0
    
    try:
        AllChem.EmbedMolecule(mol, randomSeed=42, useRandomCoords=True)
        AllChem.MMFFOptimizeMolecule(mol)
        features['PMI1_3D'] = Descriptors3D.PMI1(mol)
        features['PMI2_3D'] = Descriptors3D.PMI2(mol)
        features['PMI3_3D'] = Descriptors3D.PMI3(mol)
    except:
        features['PMI1_3D'] = 0
        features['PMI2_3D'] = 0
        features['PMI3_3D'] = 0
    
    features['HBond_Potential'] = features['NumHDonors'] * features['NumHAcceptors']
    features['Polar_Surface_per_Atom'] = features['TPSA'] / max(features['NumAtoms'], 1)
    features['Molecular_Density'] = features['MolWt'] / max(features['LabuteASA'], 1)
    features['Ring_Complexity'] = features['RingCount'] * features['NumAromaticRings']
    features['Hetero_Ratio'] = features['NumHeteroatoms'] / max(features['NumHeavyAtoms'], 1)
    features['Branching_Index'] = features['Kappa1'] / max(features['NumAtoms'], 1)
    
    fp_morgan_2048 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    for idx, bit in enumerate(fp_morgan_2048):
        features[f'morgan2048_{idx}'] = int(bit)
    
    fp_morgan_radius3 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=1024)
    for idx, bit in enumerate(fp_morgan_radius3):
        features[f'morgan_r3_{idx}'] = int(bit)
    
    maccs = MACCSkeys.GenMACCSKeys(mol)
    for idx, bit in enumerate(maccs):
        features[f'maccs_{idx}'] = int(bit)
    
    fp_rdkit = Chem.RDKFingerprint(mol, fpSize=1024)
    for idx, bit in enumerate(fp_rdkit):
        features[f'rdkit_fp_{idx}'] = int(bit)
    
    return features

print("Extracting molecular features from train set...")
train_mol_features = train['SMILES'].apply(extract_molecular_features)
train_mol_df = pd.DataFrame(list(train_mol_features))

print("Extracting molecular features from test set...")
test_mol_features = test['SMILES'].apply(extract_molecular_features)
test_mol_df = pd.DataFrame(list(test_mol_features))

train_mol_df = train_mol_df.fillna(0)
test_mol_df = test_mol_df.fillna(0)

print(f"Extracted {len(train_mol_df.columns)} molecular features")

X = train_mol_df
y = train['Tm']
X_test = test_mol_df

zero_var_cols = X.columns[X.std() == 0].tolist()
if zero_var_cols:
    print(f"Removing {len(zero_var_cols)} zero-variance features")
    X = X.drop(columns=zero_var_cols)
    X_test = X_test.drop(columns=zero_var_cols)

print(f"Final feature count: {X.shape[1]}")

class RegressionTrainer:
    def __init__(self, model, n_folds=5, random_state=42):
        self.model = model
        self.n_folds = n_folds
        self.random_state = random_state
        self.kfold = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
        
    def train_predict(self, X_train, y_train, X_test, verbose=True):
        oof_preds = np.zeros(len(X_train))
        test_preds = np.zeros(len(X_test))
        
        for fold, (train_idx, val_idx) in enumerate(self.kfold.split(X_train), 1):
            if verbose:
                print(f"Fold {fold}/{self.n_folds}...", end=" ")
            
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            
            fold_model = clone(self.model)
            
            if hasattr(fold_model, 'fit') and 'eval_set' in inspect.signature(fold_model.fit).parameters:
                fit_params = {'eval_set': [(X_val, y_val)]}
                
                if 'verbose' in inspect.signature(fold_model.fit).parameters:
                    fit_params['verbose'] = False
                
                fold_model.fit(X_tr, y_tr, **fit_params)
            else:
                fold_model.fit(X_tr, y_tr)
            
            oof_preds[val_idx] = fold_model.predict(X_val)
            test_preds += fold_model.predict(X_test) / self.n_folds
            
            fold_mae = mean_absolute_error(y_val, oof_preds[val_idx])
            if verbose:
                print(f"MAE: {fold_mae:.4f}")
        
        overall_mae = mean_absolute_error(y_train, oof_preds)
        if verbose:
            print(f"Overall OOF MAE: {overall_mae:.4f}\n")
        
        return oof_preds, test_preds, overall_mae

print("\nTraining XGBoost...")
xgb_model = XGBRegressor(
    n_estimators=5000,
    max_depth=10,
    learning_rate=0.005,
    subsample=0.8,
    colsample_bytree=0.7,
    min_child_weight=3,
    gamma=0.1,
    reg_alpha=0.5,
    reg_lambda=1.0,
    random_state=42,
)

trainer = RegressionTrainer(xgb_model, n_folds=10)
oof_xgb, test_xgb, mae_xgb = trainer.train_predict(X, y, X_test)

print("\nTraining LightGBM...")
lgb_model = LGBMRegressor(
    n_estimators=5000,
    num_leaves=128,
    learning_rate=0.005,
    subsample=0.8,
    colsample_bytree=0.7,
    min_child_samples=20,
    reg_alpha=0.5,
    reg_lambda=1.0,
    random_state=42,
    force_row_wise=True,
    verbosity=-1
)

trainer = RegressionTrainer(lgb_model, n_folds=10)
oof_lgb, test_lgb, mae_lgb = trainer.train_predict(X, y, X_test)

print("\nTraining CatBoost...")
cat_model = CatBoostRegressor(
    iterations=5000,
    depth=10,
    learning_rate=0.005,
    l2_leaf_reg=5,
    random_state=42,
    verbose=False
)

trainer = RegressionTrainer(cat_model, n_folds=10)
oof_cat, test_cat, mae_cat = trainer.train_predict(X, y, X_test)

oof_stack = np.column_stack([oof_xgb, oof_lgb, oof_cat])
test_stack = np.column_stack([test_xgb, test_lgb, test_cat])

best_alpha = None
best_mae = float('inf')

print("\nOptimizing Ridge ensemble:")
for alpha in [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]:
    ridge = Ridge(alpha=alpha, random_state=42)
    ridge.fit(oof_stack, y)
    ridge_oof = ridge.predict(oof_stack)
    mae = mean_absolute_error(y, ridge_oof)
    
    print(f"Alpha {alpha:7.3f}: MAE = {mae:.4f}")
    
    if mae < best_mae:
        best_mae = mae
        best_alpha = alpha

print(f"\nBest alpha: {best_alpha} (MAE: {best_mae:.4f})")

final_ridge = Ridge(alpha=best_alpha, random_state=42)
final_ridge.fit(oof_stack, y)
ensemble_test = final_ridge.predict(test_stack)

print("\nEnsemble Weights:")
model_names = ['XGBoost', 'LightGBM', 'CatBoost']
for name, coef in zip(model_names, final_ridge.coef_):
    print(f"{name}: {coef:.4f}")
print(f"Intercept: {final_ridge.intercept_:.4f}")

submission = pd.DataFrame({
    'id': test['id'],
    'Tm': ensemble_test
})

submission.to_csv('submission.csv', index=False)

print(f"\nExpected CV Score: {best_mae:.4f}")
print(f"Prediction range: {ensemble_test.min():.1f}K to {ensemble_test.max():.1f}K")
print(f"Training range: {y.min():.1f}K to {y.max():.1f}K")
print("\nSubmission saved as: submission.csv")
print("\nFirst 10 predictions:")
print(submission.head(10))

Extracting molecular features from train set...


[06:56:38] 

****
Pre-condition Violation
molecule has no conformers
Violation occurred on line 208 in file /project/build/temp.linux-x86_64-cpython-312/rdkit/Code/GraphMol/Descriptors/PMI.cpp
Failed Expression: mol.getNumConformers() >= 1
----------
Stacktrace:
----------
****

[06:56:38] DEPRECATION WARNING: please use MorganGenerator
[06:56:38] DEPRECATION WARNING: please use MorganGenerator
[06:56:38] 

****
Pre-condition Violation
molecule has no conformers
Violation occurred on line 208 in file /project/build/temp.linux-x86_64-cpython-312/rdkit/Code/GraphMol/Descriptors/PMI.cpp
Failed Expression: mol.getNumConformers() >= 1
----------
Stacktrace:
----------
****

[06:56:38] Molecule does not have explicit Hs. Consider calling AddHs()
[06:56:38] Molecule does not have explicit Hs. Consider calling AddHs()
[06:56:38] DEPRECATION WARNING: please use MorganGenerator
[06:56:38] DEPRECATION WARNING: please use MorganGenerator
[06:56:38] 

****
Pre-condition Violation
molecule has no co

Extracting molecular features from test set...


[06:57:22] 

****
Pre-condition Violation
molecule has no conformers
Violation occurred on line 208 in file /project/build/temp.linux-x86_64-cpython-312/rdkit/Code/GraphMol/Descriptors/PMI.cpp
Failed Expression: mol.getNumConformers() >= 1
----------
Stacktrace:
----------
****

[06:57:22] Molecule does not have explicit Hs. Consider calling AddHs()
[06:57:22] Molecule does not have explicit Hs. Consider calling AddHs()
[06:57:22] DEPRECATION WARNING: please use MorganGenerator
[06:57:22] DEPRECATION WARNING: please use MorganGenerator
[06:57:22] 

****
Pre-condition Violation
molecule has no conformers
Violation occurred on line 208 in file /project/build/temp.linux-x86_64-cpython-312/rdkit/Code/GraphMol/Descriptors/PMI.cpp
Failed Expression: mol.getNumConformers() >= 1
----------
Stacktrace:
----------
****

[06:57:22] Molecule does not have explicit Hs. Consider calling AddHs()
[06:57:22] Molecule does not have explicit Hs. Consider calling AddHs()
[06:57:22] DEPRECATION WARNING: pl

Extracted 4317 molecular features
Removing 139 zero-variance features
Final feature count: 4178

Training XGBoost...
Fold 1/10... MAE: 28.7327
Fold 2/10... MAE: 26.9203
Fold 3/10... MAE: 28.6026
Fold 4/10... MAE: 26.3331
Fold 5/10... MAE: 29.3242
Fold 6/10... MAE: 27.3803
Fold 7/10... MAE: 29.9419
Fold 8/10... MAE: 25.1103
Fold 9/10... MAE: 25.4813
Fold 10/10... MAE: 29.3818
Overall OOF MAE: 27.7209


Training LightGBM...
Fold 1/10... MAE: 30.4638
Fold 2/10... MAE: 27.6530
Fold 3/10... MAE: 28.7903
Fold 4/10... MAE: 26.9556
Fold 5/10... MAE: 29.7128
Fold 6/10... MAE: 29.9813
Fold 7/10... MAE: 30.8310
Fold 8/10... MAE: 24.9328
Fold 9/10... MAE: 27.2451
Fold 10/10... MAE: 30.1015
Overall OOF MAE: 28.6670


Training CatBoost...
Fold 1/10... MAE: 30.8046
Fold 2/10... MAE: 27.7376
Fold 3/10... MAE: 30.2960
Fold 4/10... MAE: 27.5294
Fold 5/10... MAE: 33.2656
Fold 6/10... MAE: 28.7500
Fold 7/10... MAE: 31.1938
Fold 8/10... MAE: 26.9587
Fold 9/10... MAE: 27.2727
Fold 10/10... MAE: 31.1913
Over